# L-shaped method using Single and Multi-cut algorithm

In [ ]:
import gurobipy as gp;
from gurobipy import GRB;
import numpy as np;
import math;
import time;

### Input data/Parameters for the problem

$\xi_1$ = 4 or 6, w.p. 1/2 each <br>
$\xi_2$ = 4 or 8, w.p. 1/2 each <br>

Therefore, possible scenarios are: $\{(4,4), (4,8), (6,4), (6,8)\}$

In [760]:
xi = [[4, 6],                   # realizations of $\xi_1$ 
      [4, 8]];                  # realizations of $\xi_2$

prob_xi= [[1/2, 1/2],           # prob of realizations for $\xi_1$ 
          [1/2, 1/2]];          # prob of realizations for $\xi_2$ 

xi_comb = [(xi[0][i],xi[1][j]) for i in range(len(xi[0])) for j in range(len(xi[1]))]; # all possible scenarios

prob_scen = [prob_xi[i][j]*prob_xi[i][j] for i in range(len(xi[0])) for j in range(len(xi[1]))]; # prob of each scenario

num_scen = len(xi_comb)

x_coeff = [3, 2]; # coefficients for x variables

y_coeff = [-15, -12]; # coefficients for y variables

## Method 1: Single-cut Algorithm

 The master problem right now is, <br>
 min $3 x_1 + 2 x_2 + \theta$ <br>
 s.t. $x \geq 0$

In [761]:
SC_MP = gp.Model("SC_Master");
SC_MP.modelsense = GRB.MINIMIZE;

x_sc = {};
for i in range(2):
    x_sc[i] = SC_MP.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = x_coeff[i]);

# Lower bound of -1e10 applied to avoid theta taking -(infinity) value for the first iteration.

theta_sc = SC_MP.addVar(vtype = GRB.CONTINUOUS, lb = -1e10, ub = 0, obj = 1.0); # single variable for single cut
    
SC_MP.setParam("OutputFlag", 0);

SC_MP.update();

The subproblem is given by, <br>
<br>
min $-15y_1 - 12y_2$ <br>
s.t. 
$y_1 \geq 0.8\xi_1----------------\pi_1$ <br>
$y_1 \leq \xi_1-------------------\pi_2$<br>
$y_2 \geq 0.8\xi_2------------------\pi_3$ <br>
$y_2 \leq \xi_2-------------------\pi_4$<br>
$3y_1 + 5y_2 \leq x_1---------------\pi_5$ <br>
$2y_1 + 5y_2 \leq x_2---------------\pi_6$ <br>

where dual are denoted by $\pi$

In [762]:
SC_SP = gp.Model("SC_Subproblem");
SC_SP.modelsense = GRB.MINIMIZE;

y_sc = {};
for i in range(2):
    y_sc[i] = SC_SP.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = y_coeff[i]);

Constr_sc = {};
Constr_sc[0] = SC_SP.addConstr(y_sc[0] >= 0.8);
Constr_sc[1] = SC_SP.addConstr(y_sc[0] <= 0);
Constr_sc[2] = SC_SP.addConstr(y_sc[1] >= 0.8);
Constr_sc[3] = SC_SP.addConstr(y_sc[1] <= 0);    
Constr_sc[4] = SC_SP.addConstr(3*y_sc[0] + 2*y_sc[1] <= 0);
Constr_sc[5] = SC_SP.addConstr(2*y_sc[0] + 5*y_sc[1] <= 0);

SC_SP.setParam("OutputFlag", 0);
SC_SP.setParam("InfUnbdInfo", 1);

SC_SP.update()

### The algorithm used for single-cut version is as follows:

1. Solve master problem without adding any cuts. <br>
2. Update the lower bound. <br> 
3. Retrieve the master problem solution $(x,\theta)$. <br>
4. Solve all the subproblems and store the termination status, objective value and dual values. <br>
5. If one or more subproblems is/are infeasible: <br>
    i. Add corresponding feasibility cut(s) to the master problem. <br>
    ii. Move to step 1. <br>
6. If all subproblems are feasible: <br>
    i. Calculate the upper bound. <br>
    ii. If gap between upper bound and lower bound is less than a specified value, exit and report the solution. <br>
    iii. Else, if theta is less than recourse value, add an optimality cut to the master problem. <br>
    iv. Go to step 1. <br>
 
 For given values of dual/extreme ray $\pi^k$: <br>
 
 Feasibility Cut: $\-pi_5^kx_1 - \pi_6^kx_2 \geq 0.8\xi_1^k \pi_1^k + \xi_1^k \pi_2^k + 0.8\xi_2^k \pi_3^k + \xi_2^k  \pi_4^k \; ,k = \{1,2,3,4\}$ if infeasible. <br>
 Optimality Cut: $\theta - \sum_{k=1}^4p_k(\pi_5^kx_1 + \pi_6^kx_2) \geq \sum_{k=1}^4(0.8\xi_1^k \pi_1^k + \xi_1^k \pi_2^k + 0.8\xi_2^k \pi_3^k + \xi_2^k  \pi_4^k)$ 

In [763]:
SC_LB = -1e10;
SC_UB = 1e10;
iter = 0;

start_time_sc = time.time();

while (SC_UB-SC_LB)*1/SC_UB > 1e-5:
    
    iter += 1; 
    
    print('\n*******************************Iteration', iter, '*******************************')
    
    SC_MP.optimize();   
    
    # store the first stage solutions
    xval_sc = np.zeros(2);
    for i in range(2):
        xval_sc[i] = x_sc[i].x;
        
    theta_val_sc = theta_sc.x;
    
    # Update the lower bound
    SC_LB = SC_MP.objVal;
    
    print('Lower Bound: ', SC_LB)
    
    # Initialize lists to store the duals/extreme ray, termination status, objective value 
    
    duals_sc = np.zeros(num_scen, dtype = np.ndarray);
    SP_status_sc = np.zeros(num_scen);
    SP_objval_sc = np.zeros(num_scen);
    
    temp_LB_sc = 0;
    
    # Update the subproblem constraints for given first stage solution 
    
    Constr_sc[4].setAttr(GRB.Attr.RHS, xval_sc[0]);
    Constr_sc[5].setAttr(GRB.Attr.RHS, xval_sc[1]);
    
    # Solve each subproblem
    
    for i in range(num_scen):
        
        # Update the subproblem constraints for a given scenario
        
        Constr_sc[0].setAttr(GRB.Attr.RHS, 0.8*xi_comb[i][0]);
        Constr_sc[1].setAttr(GRB.Attr.RHS, xi_comb[i][0]);
        Constr_sc[2].setAttr(GRB.Attr.RHS, 0.8*xi_comb[i][1]);
        Constr_sc[3].setAttr(GRB.Attr.RHS, xi_comb[i][1]);
           
        SC_SP.update();
        
        SC_SP.optimize();
        
        #If infeasible or optimal, store the duals/extreme ray, termination status, objective value 
            
        SP_status_sc[i] = SC_SP.status; 
    
        if SC_SP.status == GRB.INFEASIBLE:
                
            SP_objval_sc[i] = math.nan;
                
            duals_sc[i] = [Constr_sc[0].farkasdual, Constr_sc[1].farkasdual, Constr_sc[2].farkasdual, 
                               Constr_sc[3].farkasdual, Constr_sc[4].farkasdual, Constr_sc[5].farkasdual];
                                             
        elif SC_SP.status == GRB.OPTIMAL:
                
            SP_objval_sc[i] = SC_SP.objVal;
                               
            duals_sc[i] = [Constr_sc[0].pi, Constr_sc[1].pi, Constr_sc[2].pi, Constr_sc[3].pi, Constr_sc[4].pi
                           , Constr_sc[5].pi];
    
    # If any subproblem is infeasible add feasibility cuts for the corresponding subproblems and proceed to next iteration
                
    if GRB.INFEASIBLE in SP_status_sc:
        
        scenr_count_sc = 0;
        
        print('Upper Bound: ', SC_UB)
        
        for i in range(num_scen):
            
            if SP_status_sc[i] == GRB.INFEASIBLE:

                fcut_rhs_sc = duals_sc[i][0] * (0.8*xi_comb[i][0]) + duals_sc[i][1] * (xi_comb[i][0]) + duals_sc[i][2] \
                                * (0.8*xi_comb[i][1]) +  duals_sc[i][3] * (xi_comb[i][1]);

                SC_MP.addConstr(duals_sc[i][4] * (x_sc[0]) + duals_sc[i][5] * (x_sc[1]) >= -fcut_rhs_sc);

                print('Added Feasibility Cut for Scenario ',scenr_count_sc, ':', duals_sc[i][4], 'x1 + '
                      ,duals_sc[i][5], 'x2 >= ', -fcut_rhs_sc); 
                    
            scenr_count_sc += 1;
            
        continue;
        
    # If all subproblems are feasible, check and if needed update upper bound, if gap is not closed \
    # add an optimality cut if theta is less than recourse value and proceed to next iteration.

    else:
                        
        temp_UB_sc = sum(xval_sc[i] * x_coeff[i] for i in range(2)) + sum(prob_scen[i] * SP_objval_sc[i] 
                                                                          for i in range(num_scen));              
                
        if temp_UB_sc < SC_UB:
                   
            SC_UB = temp_UB_sc;
            
        print('Upper Bound: ', SC_UB)
                
        if (SC_UB-SC_LB)*1.0/SC_UB < 1e-5:
                    
            print('\n\t\t<<<<Model converged within the required gap.>>>>')
                   
            break;
                
        else: 
                
            if theta_val_sc < sum(prob_scen[i]*SP_objval_sc[i] for i in range(num_scen)) - 1e-5:                      

                ocut_rhs_sc = sum(prob_scen[i]*(duals_sc[i][0] * (0.8*xi_comb[i][0]) + duals_sc[i][1] * (xi_comb[i][0]) \
                                + duals_sc[i][2] * (0.8*xi_comb[i][1]) +  duals_sc[i][3] * (xi_comb[i][1]))
                                  for i in range(num_scen));

                SC_MP.addConstr(theta_sc - sum(prob_scen[i]*(duals_sc[i][4] * (x_sc[0]) + duals_sc[i][5] \
                                                              * (x_sc[1])) for i in range(num_scen)) >= ocut_rhs_sc);

                print('Added Optimality Cut: theta', - sum(prob_scen[i]*duals_sc[i][4] for i in range(num_scen))
                          ,'x1' , - sum(prob_scen[i]*duals_sc[i][5] for i in range(num_scen)), 'x2 >=' ,ocut_rhs_sc);

end_time_sc = time.time();

print('\nx1 = %f, x2 = %f, Total Iterations = %d, Execution Time = %f seconds' % (xval[0], xval[1], iter, end_time_sc-start_time_sc));


*******************************Iteration 1 *******************************
Lower Bound:  -10000000000.0
Upper Bound:  10000000000.0
Added Feasibility Cut for Scenario  0 : 0.5 x1 +  0.0 x2 >=  8.0
Added Feasibility Cut for Scenario  1 : 0.5 x1 +  0.0 x2 >=  11.200000000000001
Added Feasibility Cut for Scenario  2 : 0.5 x1 +  0.0 x2 >=  10.400000000000002
Added Feasibility Cut for Scenario  3 : 0.5 x1 +  0.0 x2 >=  13.600000000000001

*******************************Iteration 2 *******************************
Lower Bound:  -9999999918.4
Upper Bound:  10000000000.0
Added Feasibility Cut for Scenario  0 : 0.0 x1 +  0.5 x2 >=  11.2
Added Feasibility Cut for Scenario  1 : 0.0 x1 +  0.5 x2 >=  19.2
Added Feasibility Cut for Scenario  2 : 0.0 x1 +  0.5 x2 >=  12.8
Added Feasibility Cut for Scenario  3 : 0.0 x1 +  0.5 x2 >=  20.8

*******************************Iteration 3 *******************************
Lower Bound:  -9999999835.2
Upper Bound:  30.939999999999998
Added Optimality Cut: theta 1

## Method 2: Multi-cut Algorithm

The master problem right now is, <br>
 min $3 x_1 + 2 x_2 + \sum_{k=1}^4p_k\theta_k$ <br>
 s.t. $x \geq 0$

In [766]:
MP = gp.Model("Master");
MP.modelsense = GRB.MINIMIZE;

x = {};
for i in range(2):
    x[i] = MP.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = x_coeff[i]);
    
theta = {};
for i in range(num_scen):
    theta[i] = MP.addVar(vtype = GRB.CONTINUOUS, lb = -1e10, ub = 0, obj = prob_scen[i]);
    
MP.setParam("OutputFlag", 0);

MP.update();

The subproblem is given by, <br>
<br>
min $-15y_1 - 12y_2$ <br>
s.t. 
$y_1 \geq 0.8\xi_1----------------\pi_1$ <br>
$y_1 \leq \xi_1-------------------\pi_2$<br>
$y_2 \geq 0.8\xi_2------------------\pi_3$ <br>
$y_2 \leq \xi_2-------------------\pi_4$<br>
$3y_1 + 5y_2 \leq x_1---------------\pi_5$ <br>
$2y_1 + 5y_2 \leq x_2---------------\pi_6$ <br>

where dual are denoted by $\pi$

In [767]:
SP = gp.Model("Subproblem");
SP.modelsense = GRB.MINIMIZE;

y = {};
for i in range(2):
    y[i] = SP.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = y_coeff[i]);

Constr = {};
Constr[0] = SP.addConstr(y[0] >= 0.8);
Constr[1] = SP.addConstr(y[0] <= 0);
Constr[2] = SP.addConstr(y[1] >= 0.8);
Constr[3] = SP.addConstr(y[1] <= 0);    
Constr[4] = SP.addConstr(3*y[0] + 2*y[1] <= 0);
Constr[5] = SP.addConstr(2*y[0] + 5*y[1] <= 0);

SP.setParam("OutputFlag", 0);
SP.setParam("InfUnbdInfo", 1);

SP.update()

### The algorithm used for multi-cut version is as follows:

1. Solve master problem without adding any cuts. <br>
2. Update the lower bound. <br> 
3. Retrieve the master problem solution $x,\theta_k \forall k$. <br>
4. Solve all the subproblems and store the termination status, objective value and dual values. <br>
5. If one or more subproblems is/are infeasible: <br>
    i. Add corresponding feasibility cut(s) to the master problem. <br>
    ii. Move to step 1. <br>
6. If all subproblems are feasible: <br>
    i. Calculate the upper bound. <br>
    ii. If gap between upper bound and lower bound is less than a specified value, exit and report the solution. <br>
    iii. Else for each scenario, if theta is less than recourse value, add an optimality cut to the master problem. <br>
    iv. Go to step 1.
    
 For given values of dual/extreme ray $\pi^k$: <br>
 
 Feasibility Cut: $\-pi_5^kx_1 - \pi_6^kx_2 \geq 0.8\xi_1^k \pi_1^k + \xi_1^k \pi_2^k + 0.8\xi_2^k \pi_3^k + \xi_2^k  \pi_4^k \; ,k = \{1,2,3,4\}$ if infeasible. <br>
 Optimality Cut: $\theta_k - \pi_5^kx_1 + \pi_6^kx_2 \geq 0.8\xi_1^k \pi_1^k + \xi_1^k \pi_2^k + 0.8\xi_2^k \pi_3^k + \xi_2^k  \pi_4^k \; ,k = \{1,2,3,4\}$ 

In [768]:
LB = 0;
UB = 1e10;
iter = 0;

start_time_mc = time.time();

while (UB-LB)*1/UB > 1e-5:
    
    iter += 1;
    
    print('\n*******************************Iteration', iter, '*******************************')
    
    MP.optimize();
    
    # store the first stage solutions
    
    xval = np.zeros(2);
    for i in range(2):
        xval[i] = x[i].x;
        
    theta_val = np.zeros(num_scen);   
    for i in range(num_scen):
        theta_val[i] = theta[i].x;
        
    # Update the lower bound
        
    LB = MP.objVal;
    
    print('Lower Bound: ', LB)
    
    # Initialize lists to store the duals/extreme ray, termination status, objective value  
    
    duals = np.zeros(num_scen, dtype = np.ndarray);
    SP_status = np.zeros(num_scen);
    SP_objval = np.zeros(num_scen);
    
    temp_UB = 0;
    
    # Update the subproblem constraints for given first stage solution
    
    Constr[4].setAttr(GRB.Attr.RHS, xval[0]);
    Constr[5].setAttr(GRB.Attr.RHS, xval[1]);
    
    # Solve each subproblem
    
    for i in range(num_scen):
        
        # Update the subproblem constraints for a given scenario
            
        Constr[0].setAttr(GRB.Attr.RHS, 0.8*xi_comb[i][0]);
        Constr[1].setAttr(GRB.Attr.RHS, xi_comb[i][0]);
        Constr[2].setAttr(GRB.Attr.RHS, 0.8*xi_comb[i][1]);
        Constr[3].setAttr(GRB.Attr.RHS, xi_comb[i][1]);
           
        SP.update();
        
        SP.optimize();
        
        #If infeasible or optimal, store the duals/extreme ray, termination status, objective value
            
        SP_status[i] = SP.status; 
    
        if SP.status == GRB.INFEASIBLE:
                
            SP_objval[i] = math.nan;
                
            duals[i] = [Constr[0].farkasdual, Constr[1].farkasdual, Constr[2].farkasdual, 
                               Constr[3].farkasdual, Constr[4].farkasdual, Constr[5].farkasdual];
                                             
        elif SP.status == GRB.OPTIMAL:
                
            SP_objval[i] = SP.objVal;
                               
            duals[i] = [Constr[0].pi, Constr[1].pi, Constr[2].pi, Constr[3].pi, Constr[4].pi, Constr[5].pi];
            
    # If any subproblem is infeasible add feasibility cuts for the corresponding subproblems
    # Proceed to next iteration
                
    if GRB.INFEASIBLE in SP_status:
        
        scenr_count = 0;
        
        print('Upper Bound: ', UB)
        
        for i in range(num_scen):
            
            if SP_status[i] == GRB.INFEASIBLE:

                fcut_rhs = duals[i][0] * (0.8*xi_comb[i][0]) + duals[i][1] * (xi_comb[i][0]) + duals[i][2] \
                                * (0.8*xi_comb[i][1]) +  duals[i][3] * (xi_comb[i][1]);

                MP.addConstr(duals[i][4] * (x[0]) + duals[i][5] * (x[1]) >= -fcut_rhs);

                print('Added Feasibility Cut for Scenario ',scenr_count, ':', duals[i][4], 'x1 + '
                      ,duals[i][5], 'x2 >= ', -fcut_rhs); 
                    
            scenr_count += 1;
            
        continue;
        
     # If all subproblems are feasible, check and if needed update upper bound. 
     # If gap is closed, stop and report solution.
     # If gap is not closed add optimality cuts for scenarios with theta less than recourse value. 
     # Proceed to next iteration.

    else:
                        
        temp_UB = sum(xval[i] * x_coeff[i] for i in range(2)) + sum(prob_scen[i] * SP_objval[i] for i in range(num_scen));              
                
        if temp_UB < UB:
                   
            UB = temp_UB;
            
        print('Upper Bound: ', UB)
                
        if (UB-LB)*1.0/UB < 1e-5:
                    
            print('\n\t\t<<<<Model converged within the required gap.>>>>')
                   
            break;
                
        else:
            
            scenr_count = 0;
            
            for i in range(num_scen): 
                
                if theta_val[i] < SP_objval[i] - 1e-5:                      

                    ocut_rhs = duals[i][0] * (0.8*xi_comb[i][0]) + duals[i][1] * (xi_comb[i][0]) \
                                + duals[i][2] * (0.8*xi_comb[i][1]) +  duals[i][3] * (xi_comb[i][1]);

                    MP.addConstr(theta[i] - duals[i][4] * (x[0]) - duals[i][5] * (x[1]) >= ocut_rhs);

                    print('Added Optimality Cut for Scenario ',scenr_count, ': theta', str(i), - duals[i][4]
                          ,'x1' , - duals[i][5], 'x2 >=' ,ocut_rhs);
                    
                scenr_count += 1;

end_time_mc = time.time()                  

print('\nx1 = %f, x2 = %f, Total Iterations = %d, Execution Time = %f seconds' % (xval[0], xval[1], iter, 
                                                                          end_time_mc-start_time_mc));


*******************************Iteration 1 *******************************
Lower Bound:  -10000000000.0
Upper Bound:  10000000000.0
Added Feasibility Cut for Scenario  0 : 0.5 x1 +  0.0 x2 >=  8.0
Added Feasibility Cut for Scenario  1 : 0.5 x1 +  0.0 x2 >=  11.200000000000001
Added Feasibility Cut for Scenario  2 : 0.5 x1 +  0.0 x2 >=  10.400000000000002
Added Feasibility Cut for Scenario  3 : 0.5 x1 +  0.0 x2 >=  13.600000000000001

*******************************Iteration 2 *******************************
Lower Bound:  -9999999918.4
Upper Bound:  10000000000.0
Added Feasibility Cut for Scenario  0 : 0.0 x1 +  0.5 x2 >=  11.2
Added Feasibility Cut for Scenario  1 : 0.0 x1 +  0.5 x2 >=  19.2
Added Feasibility Cut for Scenario  2 : 0.0 x1 +  0.5 x2 >=  12.8
Added Feasibility Cut for Scenario  3 : 0.0 x1 +  0.5 x2 >=  20.8

*******************************Iteration 3 *******************************
Lower Bound:  -9999999835.2
Upper Bound:  30.939999999999998
Added Optimality Cut for Scen